In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
np.set_printoptions(precision=2)
pd.set_option('precision', 2)
np.set_printoptions(suppress=True)
sns.set(style='whitegrid')

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

# Any results you write to the current directory are saved as output.

In [ ]:
input_path = 'data/train.csv'
iowa_house_data = pd.read_csv(input_path, index_col='Id')
target = iowa_house_data.iloc[:,-1].values

numerical_features = iowa_house_data.select_dtypes(include=[np.number])
print('Number of numerical features : ' + str(numerical_features.columns.values.size))
print(numerical_features.columns.values)

In [ ]:
corr = numerical_features.corr()
print('Features that have Highest Positive Correlation with SalePrice :')
print(corr.SalePrice.sort_values(ascending=False)[:10])

numeric_predictors = corr.iloc[:-1,:-1]
threshold = 0.5

imp_num_features = (numeric_predictors[abs(numeric_predictors) > threshold]\
                    [numeric_predictors != 1.0])\
                    .unstack().dropna().to_dict()
    
'''
imp_num_features contains Duplicates keys such as 
('OverallQual', 'YearBuilt') & ('YearBuilt', 'OverallQual').
DELETE THEM using Set operation below
'''
unique_imp_num_features = pd.DataFrame(list(set([(tuple(sorted(key)), imp_num_features[key]) \
                          for key in imp_num_features])), \
                                       columns=['Attribute Pair', 'Correlation'])
unique_imp_num_features = unique_imp_num_features.ix[abs\
                        (unique_imp_num_features['Correlation']).argsort()[::-1]]
unique_imp_num_features

In [ ]:
'''
    Remove features with high collinearity.
    1. (GarageArea, GarageCars)	0.88 : Remove GarageArea as this column contains 
    big numbers.
    2. (GrLivArea, TotRmsAbvGrd)	0.83 : Remove GrLivArea as this column contains
    big numbers.
    3. (1stFlrSF, TotalBsmtSF)	0.82 : Remove 1stFlrSF
    4. Finally, 6 numberical features are selected.
    OverallQual, GarageCars, TotalBsmtSF, FullBath, TotRmsAbvGrd, YearBuilt
'''